**#LOAD LIBRARIES**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns #data vizualization

# Input data files are available in the "../input/" directory.

import os
for dirname, _, filenames in os.walk('/content/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


**#Read in Data**

In [ ]:
#read in the data dictionary  
data_dictionary = pd.ExcelFile('/content/User Guide (1).xlsx')

In [ ]:
# Print sheet names
#print(data_dictionary.sheet_names)
#['Data Dictionary', 'App A1', 'App A2', 'App A3', 'App A4']

#parse app1 and app2 
app1 = data_dictionary.parse('App A1', skiprows=[0,1], names=['MA', 'State','CFS_AREA','MA_TYPE','MA_Description'])
app2 = data_dictionary.parse('App A2', skiprows=[0], names=['NAICS', 'NAICS_Description'])
app1.head()
app2.head()

In [ ]:
#read in app 3 and fill down file 
app3 = data_dictionary.parse('App A3', skiprows=[0], names=['Commodity_Code', 'Commodity_Description','Commodity_Group'])
app3['Commodity_Group'] = app3['Commodity_Group'].ffill() #need to fill down the page 
app3['Commodity_Code'] = app3['Commodity_Code'].apply(lambda x: str(x))

In [ ]:
#app4 needs to be constructed from scratch 
# initialize list of lists 
data = [['02', 'Single Mode'], 
        ['03', 'Truck'],
        ['04', 'For-hire Truck'],
        ['05', 'Private Truck'],
        ['06', 'Rail'],
        ['07', 'Water'],
        ['08', 'Inland Water'],
        ['09', 'Great Lakes'],
        ['10', 'Deep Sea'],
        ['101', 'Multiple Waterways'],
        ['11', 'Air'],
        ['12', 'Pipeline'],
        ['13', 'Multiple Mode'],
        ['14', 'Parcel-USPS-Courier'],
        ['20', 'Non-parcel multimode'],
        ['15', 'Truck and Rail'],
        ['16', 'Truck and Water'],
        ['17', 'Rail and Water'],
        ['18', 'Other Multiple Mode'],
        ['09', 'Other Mode'],
        ['00', 'Mode Suppressed']] 
  

In [ ]:
# Create the pandas DataFrame 
app4 = pd.DataFrame(data, columns = ['Mode_Code', 'Mode_Description'])
app3.head()

In [ ]:
#read in the state names data for short hand reference if needed
states = pd.read_csv('/content/state_code_to_name (1).csv')
states.head()

In [ ]:
#read in the flat file for commodity flow survey 
cfs = pd.read_csv('/content/DB (2).csv')
cfs.head()

**#EXPLORATORY DATA ANALYSIS**

In [ ]:
cfs.columns

In [ ]:
cfs.isnull().sum()

In [ ]:
cfs['EXPORT_YN'].value_counts()
cfs['EXPORT_CNTRY'].value_counts()

In [ ]:
cfs.drop([
    'EXPORT_YN',
    'EXPORT_CNTRY'
], axis = 1, inplace = True)
cfs.head()

In [ ]:
#Analyzing the summary statistics

cfs.describe(include='all')
cfs.median()

In [ ]:
cfs.shape

**#CREATING THE CSV FINAL DATASET**

In [ ]:
#rename orig state 
cfs = pd.merge(cfs, states, how='left',left_on='ORIG_STATE', right_on='StateCode')
cfs = cfs.rename(columns = {"StateName":"ORIG_STATE_NAME"}) 
del cfs['StateCode']
del cfs['ORIG_STATE']

cfs.head()

In [ ]:
#rename destination state
cfs = pd.merge(cfs, states, how='left',left_on='DEST_STATE', right_on='StateCode')
cfs = cfs.rename(columns = {"StateName":"DEST_STATE_NAME"}) 
del cfs['StateCode']
del cfs['DEST_STATE']

cfs.head()

In [ ]:
#replace NAICS with names
cfs = pd.merge(cfs, app2, how='left',left_on='NAICS', right_on='NAICS')
cfs = cfs.rename(columns = {"NAICS_Description":"INDUSTRY_NAME", "NAICS": "INDUSTRY_CODE"}) 

cfs.head(

In [ ]:
#Merge the commodity code with description
app3['Commodity_Code']=app3['Commodity_Code'].astype(int)
cfs = pd.merge(cfs, app3, how='left',left_on='SCTG', right_on='Commodity_Code')
cfs = cfs.rename(columns = {"Commodity_Description":"COMMODITY_DESCRIPTION"}) 
del cfs['SCTG']

cfs.head()

In [ ]:
# Adding the mode of shipment
app4['Mode_Code']=app4['Mode_Code'].astype(int)
cfs = pd.merge(cfs, app4, how='left',left_on='MODE', right_on='Mode_Code')
cfs = cfs.rename(columns = {"Mode_Description":"MODE OF SHIPPING"}) 
del cfs['MODE']


cfs.head()

In [ ]:
#Rename certain column names
cfs = cfs.rename({'ORIG_MA': 'ORIG_METRO_AREA', 'ORIG_CFS_AREA': 'ORIG_COMMODITY_AREA', 'DEST_MA': 'DEST_METRO_AREA', 'DEST_CFS_AREA': 'DEST_COMMODITY_AREA', 'SHIPMT_DIST_GC': 'SHIPMT_DIST_ORIG-DEST', 'WGT_FACTOR': 'SHIPMT_WGT_FACTOR' }, axis=1) 
cfs.head()

In [ ]:
#Download file as CSV
cfs.to_csv(r'/content/Whirlpool DB.csv', index = False)

**#READING THE FINAL DATASET **

In [ ]:
Final_DB = pd.read_csv('/content/Whirlpool DB.csv')
Final_DB.head()

**Question 1: Which are the American States with the most exchanges of products based on weight?**

In [ ]:
shipments = (Final_DB.groupby('DEST_STATE_NAME')
.agg({'Commodity_Code':'count', 'SHIPMT_WGHT': 'sum'})
.reset_index().sort_values(by= 'Commodity_Code', ascending= False))
shipments.head()

**QUESTION 2: Which Industries ship more goods across the US?**

In [ ]:
Industries = Final_DB.groupby(['INDUSTRY_NAME'])['Commodity_Code'].count().reset_index().sort_values(by= 'Commodity_Code', ascending= False)
Industries.head()

**Question 3: What type of goods are the most shipped across the US?**

In [ ]:
goods = Final_DB.groupby(['COMMODITY_DESCRIPTION'])['Commodity_Code'].count().reset_index().sort_values(by= 'Commodity_Code', ascending= False)
goods.head()

**Question 4: Knowing that Whirlpool and KitchenAid belong to the Industrial Classification "Electrical equipment, appliance, and component manufacturing", and we mainly ship "Electronic and Other Electrical Equipment and Components, and Office Equipment" how this subset compares with the rest?**

In [ ]:
cfs1 = Final_DB[(Final_DB['Commodity_Group'] == '35-38')]
cfs1.head()


KitchenAid = (cfs1.groupby('COMMODITY_DESCRIPTION')
.agg({'Commodity_Code':'count', 'SHIPMT_VALUE': 'mean'})
.reset_index().sort_values(by= 'Commodity_Code', ascending= False))
KitchenAid.head()

**Question 5: What are the drivers of the shipment cost given the dataset provided?**


In [ ]:
#Finding the correlation Using Pearsons Coeff

import matplotlib.pyplot as plt
plt.figure(figsize=(12,10))
cor =   Final_DB.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
#Correlation with output variable
cor_target = abs(cor["SHIPMT_COST"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.3]
relevant_features

**Question 6: What could be some metrics to compare the various industries? explain how they are defined and why they are important**

In [ ]:
shipmt_by_country = Final_DB.groupby( by = ['INDUSTRY_NAME']).agg({'Commodity_Code':'count','SHIPMT_VALUE': 'mean', 'SHIPMT_COST': 'sum', 'SHIPMT_WGHT': 'mean'}).reset_index().sort_values(by= 'SHIPMT_VALUE',ascending= False)
shipmt_by_country.head()

In [ ]:
#Finding the most used means of transport per industry

mode_of_trans = Final_DB.groupby( by = ['INDUSTRY_NAME', 'MODE OF SHIPPING'])['Mode_Code'].count().reset_index().sort_values(by= 'Mode_Code',ascending= False)
mode_of_trans.head()

#We can observe that most of the industries use FOR-HIRE TRUCK as the means of transport

**Question 7: How KitchenAid industry compares to the others based on the metrics defined before**

In [ ]:
shipmt_by_KitchenAid = cfs1.groupby(['COMMODITY_DESCRIPTION']).agg({'Commodity_Code':'count','SHIPMT_VALUE': 'mean', 'SHIPMT_COST': 'sum', 'SHIPMT_WGHT': 'mean'}).reset_index().sort_values(by= 'SHIPMT_VALUE',ascending= False)
shipmt_by_KitchenAid.head()

In [ ]:
#Even for KitchenAid we can observe that FOR-HIRE TRUCK are the most used means of transport

mode_of_trans1 = cfs1.groupby( by = ['COMMODITY_DESCRIPTION', 'MODE OF SHIPPING'])['Mode_Code'].count().reset_index().sort_values(by= 'Mode_Code',ascending= False)
mode_of_trans1.head()

                                          ********************************************************************************


